<a href="https://colab.research.google.com/github/KulsoomA20/DLRL_Umme_Kulsoom_A/blob/main/RNN/ext_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical

# Original text data
text = ("The beautiful girl whom I met last time is very intelligent also. "
       "The handsome boy whom I met yesterday is extremely clever and kind.")
chars = sorted(list(set(text)))
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

# Parameters
seq_length = 6
text_len = 50

# Prepare sequences and labels
sequences = []
labels = []
for i in range(len(text) - seq_length):
    seq = text[i:i + seq_length]
    label = text[i + seq_length]
    sequences.append([char_to_index[char] for char in seq])
    labels.append(char_to_index[label])

X = np.array(sequences)
y = to_categorical(labels, num_classes=len(chars))

# One-hot encode input
X_one_hot = to_categorical(X, num_classes=len(chars))

# RNN model
model = Sequential([
    Bidirectional(SimpleRNN(64, return_sequences=True, activation='tanh'), input_shape=(seq_length, len(chars))),
    Dropout(0.2),
    SimpleRNN(64, activation='tanh'),
    Dropout(0.2),
    Dense(len(chars), activation='softmax')
])

# Use RMSprop with gradient clipping for stability
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X_one_hot, y, epochs=50, batch_size=16)

# Temperature-based sampling function
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

# Generate text
start_seq = "The handsome"
generated_text = start_seq

for _ in range(text_len):
    x = np.array([[char_to_index[char] for char in generated_text[-seq_length:]]])
    x_one_hot = to_categorical(x, num_classes=len(chars))
    preds = model.predict(x_one_hot, verbose=0)[0]
    next_index = sample(preds, temperature=0.8)
    next_char = index_to_char[next_index]
    generated_text += next_char

print("Generated Text:")
print(generated_text)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 6, 128)         │        11,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 26)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,690 (100.35 KB)

 Trainable params: 25,690 (100.35 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0873 - loss: 3.4909
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1751 - loss: 2.5961 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4628 - loss: 1.8279 
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5743 - loss: 1.3771
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6242 - loss: 1.2985 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6904 - loss: 1.0055
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7811 - loss: 0.7409 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8428 - loss: 0.6553 
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9090 - loss: 0.4697
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8794 - loss: 0.4222
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9332 - loss: 0.3489
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9245 - loss: 0.2843
Ep